## Markov Chains

author: Jacob Schreiber <br>
contact: jmschreiber91@gmail.com

Markov chains are the simplest probabilistic model describing a sequence of observations. Essentially, for an n-th order Markov chain, each observation is modeled as $P(X_{t} | X_{t-1}, ..., X_{t-n})$ and the probability of the entire sequence is the product of these probabilities for each observation. Naturally, the first observation in the sequence cannot be modeled as this conditional distribution and so is usually modeled as a marginal distribution. The remaining $n-1$ observations also cannot be modeled as this full conditional distribution and so are modeled by smaller distributions.

These chains are easy to implement and use in pomegranate.

In [11]:
%pylab inline
import seaborn; seaborn.set_style('whitegrid')
import torch

%load_ext watermark
%watermark -m -n -p torch,torchegranate

Populating the interactive namespace from numpy and matplotlib
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
torch        : 1.13.1
torchegranate: 0.3.2

Compiler    : GCC 11.2.0
OS          : Linux
Release     : 4.15.0-142-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit



/home/jmschr/anaconda3/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['gamma', 'uniform', 'exponential', 'normal', 'poisson', 'lognormal']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


### Initialization and Fitting

Initializing a Markov chain is simple. If you have fit distributions, you can pass them in and then use the model for inferene. If you do not have fit distributions, you can specify the `k` parameter, which is the number of previous observations that the probability of each observation is conditioned on. pomegranate will automatically construct the first `k-1` distributions as well as the main conditional distribution.

In [4]:
from torchegranate.markov_chain import MarkovChain

model = MarkovChain(k=3)

TypeError: 'NoneType' object is not iterable

The model can then be fit to data using the `fit` function. However, this data must be three dimensional, with the dimensions being `(n_samples, dimensions, length)`. Most other models in pomegranate only use the first two. This does mean that Markov chains can be multivariate but a multivariate model will assume each of the dimensions are independent of each other.

In [10]:
X = torch.tensor([[[1, 0, 0, 1]], 
                  [[0, 1, 0, 0]],
                  [[0, 0, 0, 0]],
                  [[0, 0, 0, 1]],
                  [[0, 1, 1, 0]]])

model.fit(X)

torch.Size([5, 1, 4])

We can then inspect the distribution and compare them to the data to ensure that they're right.

In [ ]:
model.distributions[0].probs[0]

In [ ]:
model.distributions[1].probs[0]

In [ ]:
model.distributions[2].probs[0]

If we wanted to fit a multivariate model all we would need to do is increase the size of the second dimension.

### Probability and Log Probability

The probability of a sequence under a Markov chain is the product of the probabilities of each observation given the previous observations. Another way of putting this is that the joint probability of a sequence with n observations $P(X_{1} ... X_{n})$ is factorized along a chain and equal to $P(X_{1}) P(X_{2} | X_{1}) \prod\limits_{t=3}^{n} P(X_{t} | X_{t-1}, X_{t-2})$ for a third order Markov chain.

If you data is multivariate, the probability of each dimension is independent and multiplied together at the end. If you would like dependencies between your dimensions, you should consider encoding a single dimension to include all combinations of observations across the features.

We can calculate the probability and log probability just as easily as other models.

In [ ]:
model.probability(X)

In [ ]:
model.log_probability(X)

### Summarization

Markov chains can perform summarization of data just like other models but that data has to have the three dimensions mentioned before. Further, each chunk of data that is summarized must have the same length. This means that if you have data with different lengths, you must either summarize them one at a time or bucket the sequences such that each bucket has all the sequences of the same length.